In [2]:
import pandas as pd

In [4]:
data = pd.read_csv('../data/raw/job_titles_to_run_june2024.csv')
columns = ['job_title','alternative_job_title','description_keywords']
data.columns = columns
data.head()

,job_title,alternative_job_title,description_keywords
0,architectural manager,design manager,"""wind energy"" OR ""offshore wind"" OR ""wind powe..."
1,architectural manager,construction manager,"""wind energy"" OR ""offshore wind"" OR ""wind powe..."
2,architectural manager,infrastructure manager,"""wind energy"" OR ""offshore wind"" OR ""wind powe..."
3,armoring technician,armor operator,"""wind energy"" OR ""offshore wind"" OR ""wind powe..."
4,armoring technician,cable operator,"""wind energy"" OR ""offshore wind"" OR ""wind powe..."


In [18]:
description_keywords = list(set(data.description_keywords))
description_keywords

['"wind energy" OR "offshore wind" OR "wind power" OR "wind project" AND "drill"',
 '"wind energy" OR "wind power" OR "wind project" AND "steel"',
 '"wind energy" OR "offshore wind" OR "wind power" OR "wind project" AND "bearing"',
 '"wind energy" OR "offshore wind" OR "wind power" OR "wind project" AND "composite" AND "blade"',
 '"wind energy" OR "offshore wind" OR "wind power" OR "wind project" AND "laser"',
 '"wind energy" OR "offshore wind" OR "wind power" OR "wind project" AND "composite"',
 '"wind energy" OR "offshore wind" OR "wind power" OR "wind project" AND "heavy"',
 '"wind energy" OR "offshore wind" OR "wind power" OR "wind project" AND "coil"',
 nan,
 '"wind energy" OR "offshore wind" OR "wind power" OR "wind project"',
 '"wind energy" OR "offshore wind" OR "wind power" OR "wind project" AND "blade" AND "test"',
 '"wind energy" OR "offshore wind" OR "wind power" OR "wind project"  AND "repair" AND "install" AND "maintain"',
 '"wind energy" OR "offshore wind" OR "wind power

In [21]:
n = 4

description_keywords = list(set(data.description_keywords))
job_titles = data['job_title'][data['description_keywords'] == description_keywords[n]]
alternative_job_titles = data['alternative_job_title'][data['description_keywords'] == description_keywords[n]]
print(description_keywords[n])
print(job_titles)
print(alternative_job_titles)

"wind energy" OR "offshore wind" OR "wind power" OR "wind project" AND "laser"
107    laser operator
108    laser operator
109    laser operator
Name: job_title, dtype: object
107      operator
108    technician
109      mechanic
Name: alternative_job_title, dtype: object


In [22]:
job_titles_norm = [job_title.lower().strip().replace(" ", "%")
                   for job_title in list(set(job_titles))]
job_titles_norm[:5]

['laser%operator']

In [23]:
alt_job_titles_norm = [job_title.lower().strip().replace(" ", "%")
                       for job_title in list(set(alternative_job_titles))]
alt_job_titles_norm[:]

['operator', 'mechanic', 'technician']

#### INDEED

In [46]:
# SELECT
fields_indeed = ["source", "object", "created_at_date", "updated_at_date",
                 "version_id", "is_deleted", "doc_id", "doc_title",
                 "doc_source_id", "doc_url", "doc_company_id",
                 "doc_company", "doc_location", "doc_description",
                 "doc_expired", "doc_indeed_origin_country", "doc_qualifications",
                 "doc_salary", "doc_benefits"]
select_clause_indeed = ','.join(fields_indeed)

# FROM: Source details
catalog = "AwsDataCatalog"
database_indeed = "julius-indeed-db"
table_indeed = "202401"
source = f'"{catalog}"."{database_indeed}"."{table_indeed}"'

# WHERE: conditions
job_title_column_name_indeed = 'doc_title'
likeness_condition_indeed = ' OR '.join([f"LOWER ({job_title_column_name_indeed}) LIKE '%{job_title}%'" for job_title in job_titles_norm])
table = "202401"
table_formatted = f'"{table}"'



condition_usa_indeed = "doc_url LIKE '%https://www.%'"


# No change
source_indeed = "indeed"
# What is  being quered 
concept = "jobposts"


external_location = f"s3://{project}/{sub_project}"
# Specify the correct filtering criteria
location = "usa"
time = "forever"

output_indeed = f"{concept}_{source_indeed}_{sub_project}_{location}_{time}_{obs}"


# SQL query for indeed
query_indeed = f'SELECT {select_clause_indeed} FROM {source} WHERE {condition_usa_indeed} AND ({likeness_condition_indeed}) AND ({condition_description}) '


# Query for indeed
query_indeed_parquet = f"""CREATE TABLE {output_indeed}
WITH (
      format = 'Parquet',
      parquet_compression = 'SNAPPY',
      external_location = '{external_location}'
) AS
{query_indeed}
"""

print(query_indeed_parquet)

CREATE TABLE jobposts_indeed_NRELIndeedMayAlternative_usa_forever_fourkeywordsindescription
WITH (
      format = 'Parquet',
      parquet_compression = 'SNAPPY',
      external_location = 's3://julius-job-classifier/NRELIndeedMayAlternative'
) AS
SELECT source,object,created_at_date,updated_at_date,version_id,is_deleted,doc_id,doc_title,doc_source_id,doc_url,doc_company_id,doc_company,doc_location,doc_description,doc_expired,doc_indeed_origin_country,doc_qualifications,doc_salary,doc_benefits FROM "AwsDataCatalog"."julius-indeed-db"."202401" WHERE doc_url LIKE '%https://www.%' AND (LOWER (doc_title) LIKE '%demand%planner%' OR LOWER (doc_title) LIKE '%receptionist%' OR LOWER (doc_title) LIKE '%budget%planner%' OR LOWER (doc_title) LIKE '%legal%researcher%' OR LOWER (doc_title) LIKE '%corporate%services%manager%' OR LOWER (doc_title) LIKE '%metal%building%engineer%' OR LOWER (doc_title) LIKE '%shipping%and%receiving%clerk%' OR LOWER (doc_title) LIKE '%transportation%coordinator%' OR LOW

In [37]:
# SELECT
fields_linkedin = ["id", "created", "last_updated", "time_posted", "title",
                   "description", "deleted",  "url", "company_id", "company_name",
                   "location", "country", "seniority", "employment_type", "salary",
                   "linkedin_job_id", "industry"]

select_clause_linkedin = ','.join(fields_linkedin)


# FROM: Source details
catalog = "AwsDataCatalog"
database_linkedin = "julius-linkedin-v3-db"
table_linkedin = "202401"
source = f'"{catalog}"."{database_linkedin}"."{table_linkedin}"'

# WHERE: conditions
job_title_column_name_linkedin = 'title'
likeness_condition_linkedin = ' OR '.join([f"LOWER ({job_title_column_name_linkedin}) LIKE '%{job_title}%'" for job_title in job_titles_norm])
table = "202401"
table_formatted = f'"{table}"'

# Conditions
condition_location = "LOWER (location) LIKE '%united%states' OR LOWER (country) LIKE '%united%states%'"

# No change
source_linkedin = "linkedin"
# What is  being quered 
concept = "jobposts"


external_location = f"s3://{project}/{sub_project}"
# Specify the correct filtering criteria
location = "usa"
time = "forever"
obs = "windindescription"
output_linkedin = f"{concept}_{source_linkedin}_{sub_project}_{location}_{time}_{obs}"


# SQL query for indeed
query_linkedin = f'SELECT {select_clause_linkedin} FROM {source} WHERE ({likeness_condition_linkedin}) AND ({condition_description}) AND ({condition_location})'


# Query for indeed
query_linkedin_parquet = f"""CREATE TABLE {output_linkedin}
WITH (
      format = 'Parquet',
      parquet_compression = 'SNAPPY',
      external_location = '{external_location}'
) AS
{query_linkedin}
"""

print(query_linkedin_parquet)

CREATE TABLE jobposts_linkedin_NRELIndeedMay_usa_forever_windindescription
WITH (
      format = 'Parquet',
      parquet_compression = 'SNAPPY',
      external_location = 's3://julius-job-classifier/NRELIndeedMay'
) AS
SELECT id,created,last_updated,time_posted,title,description,deleted,url,company_id,company_name,location,country,seniority,employment_type,salary,linkedin_job_id,industry FROM "AwsDataCatalog"."julius-linkedin-v3-db"."202401" WHERE (LOWER (title) LIKE '%administrative%services%manager%' OR LOWER (title) LIKE '%deputy%facilities%manager%' OR LOWER (title) LIKE '%bearings%design%engineer%' OR LOWER (title) LIKE '%blasting%technician%' OR LOWER (title) LIKE '%communications%manager%' OR LOWER (title) LIKE '%director%of%quality%and%operations%support%' OR LOWER (title) LIKE '%electrical%engineer%' OR LOWER (title) LIKE '%engineering%manager%' OR LOWER (title) LIKE '%freight%forwarder%' OR LOWER (title) LIKE '%heavy-lift%crane%operator%' OR LOWER (title) LIKE '%industrial%t